In [ ]:
"""
Code adapted from https://github.com/iamalexkorotin/NeuralOptimalTransport

MIT License

Copyright (c) 2023 Alexander

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

In [ ]:
import gc

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from tqdm.notebook import tqdm as tqdm

from src.distributions import MoonsSampler, StandardNormalSampler, StandardNormalScaler
from src.plotters import plot_bar_and_stochastic_2D, plot_generated_2D
from src.tools import freeze, unfreeze

In [ ]:
DEVICE_IDS = [0]

T_ITERS = 10
f_LR, T_LR = 1e-4, 1e-4

ZD = 2
Z_STD = 0.1

BATCH_SIZE = 64
Z_SIZE = 4

PLOT_INTERVAL = 100
MAX_STEPS = 10001
SEED = 0x000000

In [ ]:
DIM = 2

assert torch.cuda.is_available()
torch.cuda.set_device(DEVICE_IDS[0])
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
X_sampler = StandardNormalSampler(dim=DIM)
Y_sampler = StandardNormalScaler(MoonsSampler())

In [ ]:
def weights_init_mlp(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")


H = 100

T = nn.Sequential(
    nn.Linear(DIM + ZD, H),
    nn.ReLU(True),
    nn.Linear(H, H),
    nn.ReLU(True),
    nn.Linear(H, H),
    nn.ReLU(True),
    nn.Linear(H, DIM),
).cuda()

f = nn.Sequential(
    nn.Linear(DIM, H),
    nn.ReLU(True),
    nn.Linear(H, H),
    nn.ReLU(True),
    nn.Linear(H, H),
    nn.ReLU(True),
    nn.Linear(H, 1),
).cuda()

T.apply(weights_init_mlp)
f.apply(weights_init_mlp)

if len(DEVICE_IDS) > 1:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    f = nn.DataParallel(f, device_ids=DEVICE_IDS)

print("T params:", np.sum([np.prod(p.shape) for p in T.parameters()]))
print("f params:", np.sum([np.prod(p.shape) for p in f.parameters()]))

In [ ]:
T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=f_LR, weight_decay=1e-10)

In [ ]:
for step in tqdm(range(MAX_STEPS)):
    # T optimization
    unfreeze(T)
    freeze(f)
    for _ in range(T_ITERS):
        T_opt.zero_grad()
        X = X_sampler.sample(BATCH_SIZE).reshape(-1, 1, DIM).repeat(1, Z_SIZE, 1)
        with torch.no_grad():
            Z = torch.randn(BATCH_SIZE, Z_SIZE, ZD, device="cuda") * Z_STD
            XZ = torch.cat([X, Z], dim=2)
        T_XZ = (
            T(XZ.flatten(start_dim=0, end_dim=1))
            .permute(1, 0)
            .reshape(DIM, -1, Z_SIZE)
            .permute(1, 2, 0)
        )

        T_loss = (
            F.mse_loss(X[:, 0], T_XZ.mean(dim=1)).mean()
            - f(T_XZ.flatten(start_dim=0, end_dim=1)).mean()
            - T_XZ.var(dim=1).mean() / Z_SIZE
        )
        T_loss.backward()
        T_opt.step()

    del T_loss, T_XZ, X, Z
    gc.collect()
    torch.cuda.empty_cache()

    # f optimization
    freeze(T)
    unfreeze(f)

    X = X_sampler.sample(BATCH_SIZE)
    with torch.no_grad():
        Z = torch.randn(BATCH_SIZE, ZD, device="cuda") * Z_STD
        XZ = torch.cat([X, Z], dim=1)
        T_XZ = T(XZ)
    Y = Y_sampler.sample(BATCH_SIZE)

    f_opt.zero_grad()
    f_loss = f(T_XZ).mean() - f(Y).mean()
    f_loss.backward()
    f_opt.step()

    del f_loss, Y, X, T_XZ, Z, XZ
    gc.collect()
    torch.cuda.empty_cache()

    if step % PLOT_INTERVAL == 0:
        clear_output(wait=True)
        print("Step", step)
        print("Plotting")
        print("(DOT map appears only on the last step)")

        fig, axes = plot_generated_2D(X_sampler, Y_sampler, T, ZD, Z_STD)
        plt.show()

        fig, axes = plot_bar_and_stochastic_2D(
            X_sampler,
            Y_sampler,
            T,
            ZD,
            Z_STD,
            plot_discrete=True if step == MAX_STEPS - 1 else False,
        )
        plt.show()

    gc.collect()
    torch.cuda.empty_cache()